Alex Hemmerlin
Comp 435 Project 1
Data Exploration
English Handwritten Characters: https://www.kaggle.com/datasets/dhruvildave/english-handwritten-characters-dataset

About the Dataset:
The dataset contains 3,410 unique images of handwritten English characters. The characters are lowercase alphabetical: a-z, uppercase alphabetical: A-Z, and numeric digits: 0-9. This results in 62 classes (26+26+10) with 55 images of each class represented in the dataset.

The class label is the character that is represented in the image (0-9, A-Z, a-z). The feature is the image which is a 1200x900 png image.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhruvildave/english-handwritten-characters-dataset")

print("Path to dataset files:", path)

c:\Users\HEMMERLINAJ20\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 13.1M/13.1M [00:00<00:00, 14.1MB/s]

Extracting files...


Path to dataset files: C:\Users\HEMMERLINAJ20\.cache\kagglehub\datasets\dhruvildave\english-handwritten-characters-dataset\versions\3
